## E78.0 Disease Modelling

In [ ]:
import pandas as pd
import glob
import os
import numpy as np
pd.set_option('display.max_colwidth', 100)
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import re

## Loading and combining all data

In [ ]:
data = pd.read_csv('/home/jovyan/amedes/credentials/GZFiles/amedes_challenge/data/interim/data_interim_essed.csv') data_extract_preproc

In [ ]:
Berlin = data[data.ZENTRUM_ID == 'BER01']
Frankfurt = data[data.ZENTRUM_ID == 'FRA01']
Hamburg = data[data.ZENTRUM_ID == 'HAM08']
Stuttgart = data[data.ZENTRUM_ID == 'STR01']

## Processing of lab results


In [ ]:
    def result_inference(x):
        result=x['Result']
        if r'(' in result:
            index=result.index(r'(')
            interval=result[index:]
            result=result[:index]
        if '--' in result:
            return 'Very low'
        elif '-' in result:
            return 'Low'
        elif '++' in result:
            return 'Very high'
        elif '+' in result:
            return 'High'
        elif 'negativ' in result.lower():
            return 'Negative'
        elif 'positiv' in result.lower():
            return 'Positive'
        return 'Normal'

In [ ]:
def lab_processing(data):
    Y=data[data['TYP']=='Y']
    # Splitting rows with tests seperated by ';' eg HKT=43.0 %; MCV=87.4 fl; MCH=27.6 pg
    Y=Y.set_index(['PATIENT_HASH', 'ZENTRUM_ID', 'PATIENT_ID', 'PAT_GEBDATUM',
           'PAT_GESCHLECHT', 'DATUM', 'TYP', 'TYP_EXT',  'ICD10',
           'SICHERHEIT'])
    Y=Y['TEXT'].str.split(';').explode().reset_index()
    Y['TEXT']=Y['TEXT'].str.replace(r'^\s+',r'')
    
     # Format eg. Eosinophils 10+ % (2 - 4)
    Y_1=Y[Y.TEXT.str.contains(r'\([.0-9\s]+-[.0-9\s]+\)')].copy()
    Y_1[['Lab_test','Result']]=Y_1['TEXT'].str.split('\s',expand=True,n=1)
    Y_1.dropna(inplace=True)
    Y_1['Inference']=Y_1['Result'].apply(result_inference,axis=1)
    
     # Format LEUKO=4.5
    Y_2=Y[Y.TEXT.str.contains(r'[a-zA-Z0-9\s]+=[a-zA-Z0-9\s]+')]
    # Seperating Lab tests to their test name and results by searching for a '='
    Y_2[['Lab_test','Result']]=Y_2['TEXT'].str.split('=',expand=True,n=1)
    Y_2['Inference']=Y_2.apply(result_inference,axis=1)
    
    # Merge the data frames of the two formats
    Y_new=pd.concat([Y_1,Y_2],axis=0)
        
    return Y_new

In [ ]:
Y_new=lab_processing(data)

In [ ]:
Y_new.head()

## Processing age

In [ ]:
    
def page(data):
    
    def DATUM_preprocessing(data):
        data['entry_year'] = data.DATUM.apply(lambda x: str(x)[6:])
        
        # dealing with typos in the Berlin dataset
        if data.iloc[0,1] == 'BER01':
            data['entry_year'] = data['entry_year'].replace('50','05').replace('90','09')
            
        def extract_year(year):
            if year<'21':
                    return '20'+year
            else:
                return '19'+year

        data['entry_year'] = data['entry_year'].apply(lambda x: extract_year(x))
        _ = data['DATUM'].apply(lambda x: x[:6]) + data['entry_year']
        data['entry_date'] = pd.to_datetime(_, format='%d.%m.%Y')
        
        return data
    
    def PAT_GEBDATUM_preprocessing(data):
  
        data['birth_year'] = data.PAT_GEBDATUM.apply(lambda x: str(x)[6:])
        data['birth_entry'] = data['entry_year'].apply(lambda x: str(x)[2:]).astype(int) - data['birth_year'].astype(int)
        
        def extract_year(row):
            if row.birth_year > '20':
                return '19' + row.birth_year
            else:
                if row.entry_year < '2000':
                    return '19' + row.birth_year
                else :
                    if row.birth_entry < 0: 
                        return '19' + row.birth_year
                    else:
                        return '20' + row.birth_year
             
                                      
        data.loc[:, 'birth_year'] = data.apply(extract_year, axis = 1)  
        
        _ = data['PAT_GEBDATUM'].apply(lambda x: x[:6]) + data['birth_year']
        data['birth_date'] = pd.to_datetime(_, format='%d.%m.%Y')
        
        data = data.drop(['birth_year', 'entry_year', 'birth_entry'], axis=1)
        
        return data
    def age( data):

        _ = data['entry_date'] - data['birth_date']
        data['age'] = (_ / np.timedelta64(1, 'Y')).round(1)
        
        return data   
    return data


In [ ]:
data=page(data)

## Extracting and encoding test results most commonly found in patients suffering from Familiäre Hypercholesterinämie

In [ ]:
#function to replace values with numbers(Can be used as ordinal values)
def Inference(row):
    if row['Inference'] =='Low':
        return 1
    if row['Inference'] =='High':
        return 1
    if row['Inference'] =='Very high':
        return 2
    if row['Inference'] =='Very low':
        return 2
    if row['Inference'] =='Negative':
        return 2
    if row['Inference'] =='Normal':
        return 0
    return 'Other'

In [ ]:
def piv(findata,Y_new):
    #drop the text columns
    findata=findata.dropna(subset=['TEXT'])
    #make the final column "A"-absent in the presence of "usschl" in any row for any disease
    findata.loc[findata['TEXT'].str.contains('usschl'), ['SICHERHEIT']] = 'A'
    #make lab results columns instead of rows
    nolab=findata[findata != 'Y']
    #y_3 contains the desired rows
    Y_3=Y_new[["PATIENT_HASH",'DATUM','Lab_test','Result','Inference']]
    #group patients text to get the rows combined(later used for filtering)
    Y_text=nolab.groupby(['PATIENT_HASH'])['TEXT'].apply(','.join).reset_index()
    #merge the text to the lab results to filter based on diseases
    text_lab=Y_3.merge(Y_text,on='PATIENT_HASH',how='left')
    #filter the result finally
    text_lab=text_lab[(text_lab['Inference']!='Normal')&(text_lab.TEXT.str.contains('Familiäre Hypercholesterinämie'))]
    #selected only 20 top lab tests
    text_interest=pd.DataFrame(text_lab.Lab_test.value_counts())
    list=text_interest.reset_index().head(20).iloc[:, 0]
    pivot=Y_3[(Y_3.Lab_test.isin(list))]
    pivot['label'] = pivot.apply (lambda row: Inference(row), axis=1)
    merge=pd.pivot_table(pivot, values = 'label', index=['PATIENT_HASH','DATUM'], columns = 'Lab_test').reset_index()
    final_with_lab=nolab.merge(merge,on=['PATIENT_HASH','DATUM'],how='left')
    #replace ICD10 na with "none"
    final_with_lab.ICD10.fillna('none',inplace=True)
    #drop values where sich is nan
    final_with_lab.dropna(subset=['SICHERHEIT'],inplace=True)
    final_with_lab=final_with_lab.fillna(0)
    return final_with_lab

In [ ]:
final_with_lab=piv(data,Y_new)

## Adding the status 1 for presence & status 0 for absence of disease

In [ ]:
def Status(final_with_lab):
    l = ['amiliäre Hyperchol', 'amiliäre hyperchol', 'amiliären Hyperchol', 'amiliären hyperchol']  
    regstr = '|'.join(l)
    final_with_lab.loc[(final_with_lab['TEXT'].str.contains(regstr)),'ICD10']='E78.01'
    final_with_lab.loc[(final_with_lab['SICHERHEIT'].isin(['G','Z'])) & (final_with_lab['ICD10'].isin(['E78.01']))& (final_with_lab['TYP'].isin(['*'])), 'Status'] = 1
    final_with_lab.Status.fillna(0,inplace=True)
    return final_with_lab

In [ ]:
final_with_lab=Status(final_with_lab)

## Adding clusters to the dataset

In [ ]:
#reading the cluster files
clusters=pd.read_csv("/home/jovyan/amedes/credentials/GZFiles/clusters.csv")

In [ ]:
def clus(clusters,final_with_lab):
     #clusters.rename({'NEW_COLUMN':'clusters'},axis=1,inplace=True)
    clusters=clusters[['PATIENT_HASH','clusters']]
    final_with_lab=final_with_lab.merge(clusters,on='PATIENT_HASH',how='left')
    return final_with_lab

In [ ]:
final_with_lab=clus(clusters,final_with_lab)

## Getting train and test set

In [ ]:
def encoding(final_with_lab):
    model=final_with_lab.drop(columns=['TYP','SICHERHEIT'])
    #mean encode ICD10 as feature
    mean_encode=model.groupby('ICD10').size()/len(model)
    model.loc[:,'ICD_mean_encode']=model['ICD10'].map(mean_encode)
    model.drop(columns=['ICD10'],inplace=True)
    
    #grouping columns from the model to obtain individual id's
    #model1
    m1=model[['PATIENT_HASH','is_E75.22', 'is_E78.01',
       'is_E78.3', 'is_E71.3', 'is_disease', 'CHOL', 'CRP', 'ERY', 'FT4', 'GGT', 'GLU', 'GPT', 'HB', 'HDL',
       'HKT', 'HS', 'LDL', 'MCH', 'MCHC', 'MCV', 'TRI', 'TRIG', 'TSH','TPO',
       'VITD25','ICD_mean_encode','Status']]
    m1=m1.groupby(['PATIENT_HASH']).sum()
    #model 2
    m2=model[['PATIENT_HASH','clusters','PAT_GESCHLECHT','ZENTRUM_ID','PATIENT_ID']]
    m2=m2.groupby(['PATIENT_HASH']).last()
    #merge the two
    model1=m1.merge(m2,on=['PATIENT_HASH'],how='left')
    #fill the clusters
    model1.clusters=model1.clusters.fillna(100)
    #make non zero or one status one:
    model1.loc[model1['Status']==2, 'Status'] = 1
    #subset the columns
    model1=model1[['CHOL', 'CRP', 'ERY', 'FT4', 'GGT', 'GLU', 'GPT', 'HB', 'HDL',
       'HKT', 'HS', 'LDL', 'MCH', 'MCHC', 'MCV', 'TRI', 'TRIG', 'TSH','TPO',
       'VITD25','ICD_mean_encode','Status','PAT_GESCHLECHT','ZENTRUM_ID',
        'clusters','is_E71.3','is_E75.22', 'is_E78.01' ,'is_E78.3','is_disease']]
    #dummy encode zentrum and pad
    model1 = pd.get_dummies(model1, columns=['ZENTRUM_ID'], drop_first=True)
    model1= pd.get_dummies(model1, columns=['PAT_GESCHLECHT'], drop_first=True)
    #train&test split by sampling
    df1 = model1[model1.Status == 0].sample(1000)
    df2 = model1[~model1.index.isin(df1.index)]
    X=df2.loc[:, df2.columns != 'Status']
    y=df2.loc[:, df2.columns == 'Status']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8675309)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test=encoding(final_with_lab)

## Running different models:

In [ ]:
def run_exps(X_train, X_test, y_train, y_test):
    '''
    Lightweight script to test many models and find winners
:param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
    dfs = []
    models = [
              ('LogReg', LogisticRegression()), 
              ('RF', RandomForestClassifier()),
              ('KNN', KNeighborsClassifier()),
              ('SVM', SVC()), 
              ('GNB', GaussianNB()),
              ('XGB', XGBClassifier())
            ]
    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
    target_names = ['no_e78', 'yes_e78']
    for name, model in models:
            kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
            cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
            clf = model.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            print(name)
            print(classification_report(y_test, y_pred, target_names=target_names))
            print(confusion_matrix(y_test, y_pred,labels=[0,1]))
    results.append(cv_results)
    names.append(name)
    this_df = pd.DataFrame(cv_results)
    this_df['model'] = name
    dfs.append(this_df)
    final = pd.concat(dfs, ignore_index=True)
    return final

In [ ]:
final=run_exps(X_train, X_test, y_train, y_test)

In [ ]:
X_train.head()

## Best model and results:

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:

def best_model(X_train, X_test, y_train, y_test,final_with_lab):
    from sklearn.inspection import permutation_importance
    #import shap
    from matplotlib import pyplot as plt


    model = RandomForestClassifier(n_estimators=100)

    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
    cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
    clf = rf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred, target_names=target_names))
    print(confusion_matrix(y_test, y_pred,labels=[0,1]))

    model.fit(X_train, y_train)
    # view the feature scores
    feature_scores = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    # Creating a seaborn bar plot

    f, ax = plt.subplots(figsize=(30, 24))
    ax = sns.barplot(x=feature_scores, y=feature_scores.index, data=final_with_lab)
    ax.set_title("Visualize feature scores of the features")
    ax.set_yticklabels(feature_scores.index)
    ax.set_xlabel("Feature importance score")
    ax.set_ylabel("Features")
    plt.show()
    score=accuracy_score(y_test,y_pred)
    
    return score


In [ ]:
Y_new.head()

In [ ]:
score

In [ ]:
score=best_model(X_train, X_test, y_train, y_test,final_with_lab)

In [ ]:
best_model(X_train, X_test, y_train, y_test,final_with_lab)

In [ ]:
pivot.head()

### Pivoting LAB RESULTS COLUMNS INSTEAD OF ROWS

In [ ]:
nolab=findata[findata != 'Y']

In [ ]:
#y_3 contains the desired rows
Y_3=Y_new[["PATIENT_HASH",'DATUM','Lab_test','Result','Inference']]
#group patients text to get the rows combined(later used for filtering)
Y_text=nolab.groupby(['PATIENT_HASH'])['TEXT'].apply(','.join).reset_index()
#merge the text to the lab results to filter based on diseases
text_lab=Y_3.merge(Y_text,on='PATIENT_HASH',how='left')
#filter the result finally
text_lab=text_lab[(text_lab['Inference']!='Normal')&(text_lab.TEXT.str.contains('Familiäre Hypercholesterinämie'))]
#selected only 20 top lab tests
text_interest=pd.DataFrame(text_lab.Lab_test.value_counts())
list=text_interest.reset_index().head(20).iloc[:, 0]

In [ ]:
pivot=Y_3[(Y_3.Lab_test.isin(list))]
pivot.head()

In [ ]:
#function to replace values with numbers(Can be used as ordinal values)
def Inference(row):
    if row['Inference'] =='Low':
        return 1
    if row['Inference'] =='High':
        return 1
    if row['Inference'] =='Very high':
        return 2
    if row['Inference'] =='Very low':
        return 2
    if row['Inference'] =='Negative':
        return 2
    if row['Inference'] =='Normal':
        return 0
    return 'Other'

In [ ]:
pivot['label'] = pivot.apply (lambda row: Inference(row), axis=1)


In [ ]:
merge=pd.pivot_table(pivot, values = 'label', index=['PATIENT_HASH','DATUM'], columns = 'Lab_test').reset_index()


In [ ]:
final_with_lab=nolab.merge(merge,on=['PATIENT_HASH','DATUM'],how='left')

In [ ]:
#replace ICD10 na with "none"
final_with_lab.ICD10.fillna('none',inplace=True)
#drop values where sich is nan
final_with_lab.dropna(subset=['SICHERHEIT'],inplace=True)
final_with_lab=final_with_lab.fillna(0)

In [ ]:
final_with_lab.head()

## we add the cluster later on &now we can just move to status

## Status

In [ ]:
final_with_lab.head()

In [ ]:
l = ['amiliäre Hyperchol', 'amiliäre hyperchol', 'amiliären Hyperchol', 'amiliären hyperchol']  
regstr = '|'.join(l)
final_with_lab.loc[(final_with_lab['TEXT'].str.contains(regstr)),'ICD10']='E78.01'

In [ ]:
final_with_lab.loc[(final_with_lab['SICHERHEIT'].isin(['G','Z'])) & (nolab['ICD10'].isin(['E78.01']))& (nolab['TYP'].isin(['*'])), 'Status'] = 1

In [ ]:
final_with_lab.Status.fillna(0,inplace=True)

In [ ]:
final_with_lab.Status.value_counts()

In [ ]:
final_with_lab.loc[(final_with_lab['TEXT'].str.contains(regstr))].TYP.value_counts()

In [ ]:
final_with_lab[(final_with_lab['TYP'].isin(['*'])) & (final_with_lab.TEXT.str.contains('Familiäre Hypercholesterinämie'))]

## adding clusterwork

In [ ]:
#reading the cluster files
clusters=pd.read_csv("/home/jovyan/amedes/credentials/GZFiles/NEW-Copy1.csv")
clusters.rename({'NEW_COLUMN':'clusters'},axis=1,inplace=True)
clusters=clusters[['PATIENT_HASH','clusters']]
clusters.clusters.unique()

In [ ]:
final_with_lab=final_with_lab.merge(clusters,on='PATIENT_HASH',how='left')
final_with_lab.head()

In [ ]:
final_with_lab.columns

In [ ]:
    model=final_with_lab.drop(columns=['TYP','SICHERHEIT'])
    #mean encode ICD10 as feature
    mean_encode=model.groupby('ICD10').size()/len(model)
    model.loc[:,'ICD_mean_encode']=model['ICD10'].map(mean_encode)
    model.drop(columns=['ICD10'],inplace=True)
    model.dtypes

In [ ]:
def encoding(final_with_lab):
    model=final_with_lab.drop(columns=['TYP','SICHERHEIT'])
    #mean encode ICD10 as feature
    mean_encode=model.groupby('ICD10').size()/len(model)
    model.loc[:,'ICD_mean_encode']=model['ICD10'].map(mean_encode)
    model.drop(columns=['ICD10'],inplace=True)
    
    #grouping columns from the model to obtain individual id's
    #model1
    m1=model[['PATIENT_HASH','is_E75.22', 'is_E78.01',
       'is_E78.3', 'is_E71.3', 'is_disease', 'CHOL', 'CRP', 'ERY', 'FT4', 'GGT', 'GLU', 'GPT', 'HB', 'HDL',
       'HKT', 'HS', 'LDL', 'MCH', 'MCHC', 'MCV', 'TRI', 'TRIG', 'TSH','TPO',
       'VITD25','ICD_mean_encode','Status']]
    m1=m1.groupby(['PATIENT_HASH']).sum()
    #model 2
    m2=model[['PATIENT_HASH','clusters','PAT_GESCHLECHT','ZENTRUM_ID','PATIENT_ID']]
    m2=m2.groupby(['PATIENT_HASH']).last()
    #merge the two
    model1=m1.merge(m2,on=['PATIENT_HASH'],how='left')
    #fill the clusters
    model1.clusters=model1.clusters.fillna(100)
    #make non zero or one status one:
    model1.loc[model1['Status']==2, 'Status'] = 1
    #subset the columns
    model1=model1[['CHOL', 'CRP', 'ERY', 'FT4', 'GGT', 'GLU', 'GPT', 'HB', 'HDL',
       'HKT', 'HS', 'LDL', 'MCH', 'MCHC', 'MCV', 'TRI', 'TRIG', 'TSH','TPO',
       'VITD25','ICD_mean_encode','Status','PAT_GESCHLECHT','ZENTRUM_ID',
        'clusters','is_E71.3','is_E75.22', 'is_E78.01' ,'is_E78.3','is_disease']]
    #dummy encode zentrum and pad
    model1 = pd.get_dummies(model1, columns=['ZENTRUM_ID'], drop_first=True)
    model1= pd.get_dummies(model1, columns=['PAT_GESCHLECHT'], drop_first=True)
    #train&test split by sampling
    df1 = model1[model1.Status == 0].sample(1000)
    df2 = model1[~model1.index.isin(df1.index)]
    X=df2.loc[:, df2.columns != 'Status']
    y=df2.loc[:, df2.columns == 'Status']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8675309)
    return X_train, X_test, y_train, y_test
    
    

In [ ]:
X_train, X_test, y_train, y_test=encoding(final_with_lab)

## drop the not needed columns

In [ ]:
model=final_with_lab.drop(columns=['TYP','SICHERHEIT'])

## MEAN ENCODE ICD CODE AS FEATURE

In [ ]:
mean_encode=model.groupby('ICD10').size()/len(model)
model.loc[:,'ICD_mean_encode']=model['ICD10'].map(mean_encode)
model.drop(columns=['ICD10'],inplace=True)

In [ ]:
model.columns

In [ ]:
model.Status.unique()

In [ ]:
m1=model[['PATIENT_HASH','is_E75.22', 'is_E78.0',
       'is_E78.3', 'is_E71.3', 'is_disease', 'CHOL', 'CRP', 'ERY', 'FT4', 'GGT', 'GLU', 'GPT', 'HB', 'HDL',
       'HKT', 'HS', 'LDL', 'MCH', 'MCHC', 'MCV', 'TRI', 'TRIG', 'TSH',
       'VITD25','ICD_mean_encode','Status']]
m1=m1.groupby(['PATIENT_HASH']).sum()
m1.shape

In [ ]:
m2=model[['PATIENT_HASH','clusters','PAT_GESCHLECHT','ZENTRUM_ID','PATIENT_ID']]
#df.groupby(['id','product']).last()
m2=m2.groupby(['PATIENT_HASH']).last()
m2.shape

In [ ]:
model1=m1.merge(m2,on=['PATIENT_HASH'],how='left')

In [ ]:
model1.clusters=model1.clusters.fillna(100)

In [ ]:
model1.Status.value_counts()

In [ ]:
model1.loc[model1['Status']==2, 'Status'] = 1


In [ ]:
model1=model1[['CHOL',
 'CRP',
 'ERY',
 'FT4',
 'GGT',
 'GLU',
 'GPT',
 'HB',
 'HDL',
 'HKT',
 'HS',
 'ICD_mean_encode',
 'LDL',
 'MCH',
 'MCHC',
 'MCV',
 'PATIENT_ID',
 'PAT_GESCHLECHT',
 'Status',
 'TRI',
 'TRIG',
 'TSH',
 'VITD25',
 'ZENTRUM_ID',
 'clusters',
 'is_E71.3',
 'is_E75.22',
 'is_E78.0',
 'is_E78.3',
 'is_disease']]

In [ ]:
model1.head()

## dummy encode zentrum &pat

In [ ]:
model1 = pd.get_dummies(model1, columns=['ZENTRUM_ID'], drop_first=True)
model1= pd.get_dummies(model1, columns=['PAT_GESCHLECHT'], drop_first=True)

## modelling

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile




In [ ]:
model1.Status.value_counts()

In [ ]:
df1 = model1[model1.Status == 0].sample(1000)
df2 = model1[~model1.index.isin(df1.index)]
X=df2.loc[:, df2.columns != 'Status']
y=df2.loc[:, df2.columns == 'Status']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8675309)

In [ ]:
def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame) -> pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
:param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    
    dfs = []
models = [
          ('LogReg', LogisticRegression()), 
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC()), 
          ('GNB', GaussianNB()),
          ('XGB', XGBClassifier())
        ]
results = []
names = []
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
target_names = ['no_e78', 'yes_e78']
for name, model in models:
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        print(confusion_matrix(y_test, y_pred,labels=[0,1]))
results.append(cv_results)
names.append(name)
this_df = pd.DataFrame(cv_results)
this_df['model'] = name
dfs.append(this_df)
final = pd.concat(dfs, ignore_index=True)
return final